# Interpreting horse racing prediction with SHAP explanation

## Overview Of The Datasets

The dataset contains the race result of 6348 local races Hong Kong. The dataset includes two tables (race.csv and run.csv), which can be joined by race_id.

The dataset can be downloaded from: 
https://www.kaggle.com/datasets/gdaley/hkracing/data

delete later...
https://github.com/codeworks-data/mvp-horse-racing-prediction/blob/master/baseline_models.ipynb

https://github.com/ethan-eplee/HorseRacePrediction
..



In [3]:
pwd

'/Users/kamanbeckypang/Documents/year2_trim2/Shapley_project/Horse_racing/horse_race_all'

In [30]:
# This code imports some libraries that you will need. 
# Python ≥3.5 is required

import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np


# Pandas for overview
import pandas as pd
import collections
from collections import Counter

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn import tree
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#models
from sklearn.svm import SVC
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from scipy.stats import randint as sp_randint
from sklearn.ensemble import BaggingClassifier,VotingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier,ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Plot setup
import matplotlib as plt

import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

## Data Preprocessing
Load the csv files
- runs.csv - contain all horses details- horse type
- race.csv contain race details e.g class/ distance/ track/ win odds

In [31]:
# load the run dataset
df_runs = pd.read_csv("runs.csv")
df_races = pd.read_csv("races.csv")
df_runs.head(2)

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,time2,time3,time4,time5,time6,finish_time,win_odds,place_odds,trainer_id,jockey_id
0,0,1,3917,10,0.0,8.00,3,AUS,Gelding,60,...,21.59,23.86,24.62,NaN,NaN,83.92,9.7,3.7,118,2
1,0,2,2157,8,0.0,5.75,3,NZ,Gelding,60,...,21.99,23.30,23.70,NaN,NaN,83.56,16.0,4.9,164,57


In [32]:
print(df_races.shape)
print(df_runs.shape)
print(df_races.columns)
print(df_runs.columns)


(6349, 37)
(79447, 37)
Index(['race_id', 'date', 'venue', 'race_no', 'config', 'surface', 'distance',
       'going', 'horse_ratings', 'prize', 'race_class', 'sec_time1',
       'sec_time2', 'sec_time3', 'sec_time4', 'sec_time5', 'sec_time6',
       'sec_time7', 'time1', 'time2', 'time3', 'time4', 'time5', 'time6',
       'time7', 'place_combination1', 'place_combination2',
       'place_combination3', 'place_combination4', 'place_dividend1',
       'place_dividend2', 'place_dividend3', 'place_dividend4',
       'win_combination1', 'win_dividend1', 'win_combination2',
       'win_dividend2'],
      dtype='object')
Index(['race_id', 'horse_no', 'horse_id', 'result', 'won', 'lengths_behind',
       'horse_age', 'horse_country', 'horse_type', 'horse_rating',
       'horse_gear', 'declared_weight', 'actual_weight', 'draw',
       'position_sec1', 'position_sec2', 'position_sec3', 'position_sec4',
       'position_sec5', 'position_sec6', 'behind_sec1', 'behind_sec2',
       'behind_sec3', '

In [33]:
df_races.head(2)

,race_id,date,venue,race_no,config,surface,distance,going,horse_ratings,prize,...,place_combination3,place_combination4,place_dividend1,place_dividend2,place_dividend3,place_dividend4,win_combination1,win_dividend1,win_combination2,win_dividend2
0,0,1997-06-02,ST,1,A,0,1400,GOOD TO FIRM,40-15,485000.0,...,6.0,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN
1,1,1997-06-02,ST,2,A,0,1200,GOOD TO FIRM,40-15,485000.0,...,4.0,NaN,12.5,47.0,33.5,NaN,5,23.5,NaN,NaN


## Merge two tables

In [34]:
# In this step I am going to convert the race dataset into run dataset according to the race_id
# onlu the useful features will be selected to convert 
# venue, config, surface,distance,going,horse_rate, prize, race_class
df_runs_copy = df_runs.copy()
df_races_copy = df_races.copy()
all_runs = df_runs_copy.merge(df_races_copy[['race_id', 'venue', 'config', 'surface', 'distance','going', 'horse_ratings', 'prize','race_class']], on='race_id', how='left')
all_runs.head(5)

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,trainer_id,jockey_id,venue,config,surface,distance,going,horse_ratings,prize,race_class
0,0,1,3917,10,0.0,8.00,3,AUS,Gelding,60,...,118,2,ST,A,0,1400,GOOD TO FIRM,40-15,485000.0,5
1,0,2,2157,8,0.0,5.75,3,NZ,Gelding,60,...,164,57,ST,A,0,1400,GOOD TO FIRM,40-15,485000.0,5
2,0,3,858,7,0.0,4.75,3,NZ,Gelding,60,...,137,18,ST,A,0,1400,GOOD TO FIRM,40-15,485000.0,5
3,0,4,1853,9,0.0,6.25,3,SAF,Gelding,60,...,80,59,ST,A,0,1400,GOOD TO FIRM,40-15,485000.0,5
4,0,5,2796,6,0.0,3.75,3,GB,Gelding,60,...,9,154,ST,A,0,1400,GOOD TO FIRM,40-15,485000.0,5


In [35]:
#After I merging the dataset, each horse has its own feature and also the respective race feature
print(all_runs.shape)
print(all_runs.columns)

(79447, 45)
Index(['race_id', 'horse_no', 'horse_id', 'result', 'won', 'lengths_behind',
       'horse_age', 'horse_country', 'horse_type', 'horse_rating',
       'horse_gear', 'declared_weight', 'actual_weight', 'draw',
       'position_sec1', 'position_sec2', 'position_sec3', 'position_sec4',
       'position_sec5', 'position_sec6', 'behind_sec1', 'behind_sec2',
       'behind_sec3', 'behind_sec4', 'behind_sec5', 'behind_sec6', 'time1',
       'time2', 'time3', 'time4', 'time5', 'time6', 'finish_time', 'win_odds',
       'place_odds', 'trainer_id', 'jockey_id', 'venue', 'config', 'surface',
       'distance', 'going', 'horse_ratings', 'prize', 'race_class'],
      dtype='object')


In [36]:
# set the unique index
all_runs.reset_index(drop=True, inplace=True)

In [37]:
# Fill in null values in catagorical variable
all_runs['horse_type'].fillna('horse', inplace=True)

# convert 0/1 in horse_gear column, which means use gear or not
# Convert '--' to 0, and everything else to 1
all_runs['horse_gear'] = np.where(all_runs['horse_gear'] == '--', 0, 1)

In [38]:
unique_values_race_id = df_runs['race_id'].value_counts().index.tolist()
unique_values_horse_id = df_runs['horse_id'].value_counts().index.tolist()
unique_values_trainer_id = df_runs['trainer_id'].value_counts().index.tolist()
unique_values_jockey_id = df_runs['jockey_id'].value_counts().index.tolist()

print("Total races : ", len(unique_values_race_id),"\nTotal horses: ", len(unique_values_horse_id))
print("Total trainers : ", len(unique_values_trainer_id),"\nTotal jockeys: ", len(unique_values_jockey_id))

Total races :  6348 
Total horses:  4405
Total trainers :  176 
Total jockeys:  186


In [39]:
# select the appropiate features for future modeling
# remove time, position_sec and behind_sec
#all_runs = all_runs.drop(['lengths_behind','position_sec1','position_sec2','position_sec3', 'position_sec4', 'position_sec5', 'position_sec6', 'behind_sec1', 'behind_sec2', 'behind_sec3', 'behind_sec4', 'behind_sec5', 'behind_sec6', 'time1','time2', 'time3', 'time4', 'time5', 'time6', 'prize'], axis=1)
#all_runs.columns

In [40]:
all_runs = all_runs.fillna(0)

In [41]:
all_runs_temp = all_runs.copy()

In [42]:
all_runs_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79447 entries, 0 to 79446
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   race_id          79447 non-null  int64  
 1   horse_no         79447 non-null  int64  
 2   horse_id         79447 non-null  int64  
 3   result           79447 non-null  int64  
 4   won              79447 non-null  float64
 5   lengths_behind   79447 non-null  float64
 6   horse_age        79447 non-null  int64  
 7   horse_country    79447 non-null  object 
 8   horse_type       79447 non-null  object 
 9   horse_rating     79447 non-null  int64  
 10  horse_gear       79447 non-null  int64  
 11  declared_weight  79447 non-null  float64
 12  actual_weight    79447 non-null  int64  
 13  draw             79447 non-null  int64  
 14  position_sec1    79447 non-null  int64  
 15  position_sec2    79447 non-null  int64  
 16  position_sec3    79447 non-null  int64  
 17  position_sec

## Feature Engineering

### Modifying IDs - 'horse_id', 'jockey_id' and 'trainer_id
- Using Ids as a feature might not be meaningful since the model will interpret the id as a numerical value, potentially leading to a reduction in model performance

- Hence, I replaced the 'horse_id', 'jockey_id' and 'trainer_id' with the percentage of podiums wons.

- The "percentage of podiums won" refers to the proportion of times a horse, jockey, or trainer has finished in the top three positions in races they have participated in.

In horse racing, the "podium" typically refers to the top three finishing positions: 1st place (winner), 2nd place (runner-up), and 3rd place (third position).

The formula to calculate the percentage of podiums won is given by:

\[ \text{Percentage of Podiums Won} = \left( \frac{\text{Number of Podium Finishes}}{\text{Total Number of Races}} \right) \times 100 \]


For example, if a horse has participated in 20 races and finished in the top three positions in 10 of them, its percentage of podiums won would be \( \frac{10}{20} \times 100 = 50\% \).

In [43]:
# create new features 
# horse_id

horse_tot_race = df_runs.groupby(['horse_id'])['result'].apply(lambda x: (x).sum()).reset_index(name='horse_tot_race')
all_runs = pd.merge(all_runs,horse_tot_race,on='horse_id',how='left')

horse_tot_place = df_runs.groupby(['horse_id'])['result'].apply(lambda x: (x <=3).sum()).reset_index(name='horse_tot_place')
all_runs=pd.merge(all_runs,horse_tot_place,on='horse_id',how='left')

# jockey_id
jockey_tot_race = df_runs.groupby(['jockey_id'])['result'].apply(lambda x: (x).sum()).reset_index(name='jockey_tot_race')
all_runs = pd.merge(all_runs,jockey_tot_race,on='jockey_id',how='left')

jockey_tot_place = df_runs.groupby(['jockey_id'])['result'].apply(lambda x: (x <=3).sum()).reset_index(name='jockey_tot_place')
all_runs = pd.merge(all_runs,jockey_tot_place,on='jockey_id',how='left')

#trainer_id
trainer_tot_race = df_runs.groupby(['trainer_id'])['result'].apply(lambda x: (x).sum()).reset_index(name='trainer_tot_race')
all_runs = pd.merge(all_runs,trainer_tot_race,on='trainer_id',how='left')

trainer_tot_place = df_runs.groupby(['trainer_id'])['result'].apply(lambda x: (x <=3).sum()).reset_index(name='trainer_tot_place')
all_runs = pd.merge(all_runs,trainer_tot_place,on='trainer_id',how='left')


In [44]:
#new horse features
all_runs['horse_place_perc']=all_runs['horse_tot_place']/all_runs['horse_tot_race']

#new jockey features
all_runs['jockey_place_perc']=all_runs['jockey_tot_place']/all_runs['jockey_tot_race']

#new trainer features
all_runs['trainer_place_perc']=all_runs['trainer_tot_place']/all_runs['trainer_tot_race']

all_runs = all_runs.drop([ 'horse_tot_place', 'horse_tot_race','jockey_tot_place', 'jockey_tot_race','trainer_tot_place','trainer_tot_race', 'trainer_id'], axis=1)

In [45]:
# Create a new column 'all_runs_finished_position' and fill it with all the finishing positions
all_runs['all_runs_finished_position'] = all_runs.groupby('horse_id')['result'].transform(lambda x: ','.join(map(str, x[::-1])))

# Then, convert into list of list,each value is integer
all_runs['all_runs_finished_position'] = all_runs['all_runs_finished_position'].apply(lambda x: list(map(int, x.split(','))))
all_runs.head(2)

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,surface,distance,going,horse_ratings,prize,race_class,horse_place_perc,jockey_place_perc,trainer_place_perc,all_runs_finished_position
0,0,1,3917,10,0.0,8.00,3,AUS,Gelding,60,...,0,1400,GOOD TO FIRM,40-15,485000.0,5,0.026432,0.073136,0.063716,"[12, 7, 8, 9, 6, 11, 11, 12, 12, 12, 8, 12, 10..."
1,0,2,2157,8,0.0,5.75,3,NZ,Gelding,60,...,0,1400,GOOD TO FIRM,40-15,485000.0,5,0.023622,0.042150,0.036792,"[8, 9, 11, 12, 9, 10, 2, 9, 10, 12, 8, 10, 10,..."


In [46]:
# Get to most frequent finish	
all_runs['most_frequent_finish'] = all_runs['all_runs_finished_position'].apply(lambda x: max(set(x), key = x.count))

# Get the top 3 most frequent finish position
all_runs['top_3_most_frequent_finish'] = all_runs['all_runs_finished_position'].apply(lambda x: ','.join(map(str, [item for item, count in Counter(x).most_common(3)])))

# Find the best 3 finish position
all_runs['best_3_values'] = all_runs['all_runs_finished_position'].apply(lambda x: sorted(set(x))[:3])

# get the probability of the getting the top 3 position
all_runs['top_3_prob'] = all_runs['all_runs_finished_position'].apply(lambda x: sum(1 for pos in x if pos <= 3) / len(x))

# Average rank position
all_runs['average_rank'] = all_runs['all_runs_finished_position'].apply(lambda x: sum(x) / len(x))

# Create a new column 'is_top_3', 0 means the horse did not won in top 3 position at that race
all_runs['is_top_3'] = all_runs['result'].apply(lambda x: 1 if x in range(1, 4) else 0)

# Reset the index of the dataframe
all_runs = all_runs.reset_index(drop=True)
all_runs.head(5)


,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,horse_place_perc,jockey_place_perc,trainer_place_perc,all_runs_finished_position,most_frequent_finish,top_3_most_frequent_finish,best_3_values,top_3_prob,average_rank,is_top_3
0,0,1,3917,10,0.0,8.00,3,AUS,Gelding,60,...,0.026432,0.073136,0.063716,"[12, 7, 8, 9, 6, 11, 11, 12, 12, 12, 8, 12, 10...",12,"12,8,7","[1, 2, 3]",0.200000,7.566667,0
1,0,2,2157,8,0.0,5.75,3,NZ,Gelding,60,...,0.023622,0.042150,0.036792,"[8, 9, 11, 12, 9, 10, 2, 9, 10, 12, 8, 10, 10,...",10,"10,8,9","[1, 2, 3]",0.187500,7.937500,0
2,0,3,858,7,0.0,4.75,3,NZ,Gelding,60,...,0.024000,0.059514,0.041787,"[9, 11, 4, 6, 8, 5, 5, 7, 7, 4, 3, 9, 5, 5, 6,...",5,"5,4,9","[1, 3, 4]",0.136364,5.681818,0
3,0,4,1853,9,0.0,6.25,3,SAF,Gelding,60,...,0.014706,0.037867,0.040311,"[14, 11, 6, 3, 10, 9, 6, 9]",6,"6,9,14","[3, 6, 9]",0.125000,8.500000,0
4,0,5,2796,6,0.0,3.75,3,GB,Gelding,60,...,0.045455,0.027462,0.027202,"[3, 2, 2, 12, 9, 5, 6, 4, 4, 6, 7, 6]",6,"6,2,4","[2, 3, 4]",0.250000,5.500000,0


In [47]:
# calculate the jockey

import pandas as pd

# Calculate placements (top 3 finishes) for each jockey
jockey_stats = all_runs.groupby('jockey_id')['result'].apply(lambda x: (x <= 3).sum()).reset_index()
jockey_stats.columns = ['jockey_id', 'Top3Finishes']

# Calculate total races for each jockey
total_races = all_runs['jockey_id'].value_counts().reset_index()
total_races.columns = ['jockey_id', 'jockey_total_races']

# Merge the two DataFrames to get placements and total races together
jockey_stats = pd.merge(jockey_stats, total_races, on='jockey_id', how='left')

# Calculate placement percentage of jockey
# It means calculatet the probability of top-three finishes to total races
jockey_stats['jockey_placement_perc'] = (jockey_stats['Top3Finishes'] / jockey_stats['jockey_total_races']) * 100


all_runs = pd.merge(all_runs, jockey_stats, on='jockey_id', how='left')
all_runs.head(10)


,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,all_runs_finished_position,most_frequent_finish,top_3_most_frequent_finish,best_3_values,top_3_prob,average_rank,is_top_3,Top3Finishes,jockey_total_races,jockey_placement_perc
0,0,1,3917,10,0.0,8.00,3,AUS,Gelding,60,...,"[12, 7, 8, 9, 6, 11, 11, 12, 12, 12, 8, 12, 10...",12,"12,8,7","[1, 2, 3]",0.200000,7.566667,0,1811,4719,38.376775
1,0,2,2157,8,0.0,5.75,3,NZ,Gelding,60,...,"[8, 9, 11, 12, 9, 10, 2, 9, 10, 12, 8, 10, 10,...",10,"10,8,9","[1, 2, 3]",0.187500,7.937500,0,509,1893,26.888537
2,0,3,858,7,0.0,4.75,3,NZ,Gelding,60,...,"[9, 11, 4, 6, 8, 5, 5, 7, 7, 4, 3, 9, 5, 5, 6,...",5,"5,4,9","[1, 3, 4]",0.136364,5.681818,0,1368,3985,34.328733
3,0,4,1853,9,0.0,6.25,3,SAF,Gelding,60,...,"[14, 11, 6, 3, 10, 9, 6, 9]",6,"6,9,14","[3, 6, 9]",0.125000,8.500000,0,71,275,25.818182
4,0,5,2796,6,0.0,3.75,3,GB,Gelding,60,...,"[3, 2, 2, 12, 9, 5, 6, 4, 4, 6, 7, 6]",6,"6,2,4","[2, 3, 4]",0.250000,5.500000,0,244,1254,19.457735
5,0,6,3296,3,0.0,1.25,3,NZ,Gelding,60,...,"[14, 12, 4, 13, 6, 14, 1, 3]",14,"14,12,4","[1, 3, 4]",0.250000,8.375000,1,587,3590,16.350975
6,0,7,911,12,0.0,9.50,3,NZ,Gelding,60,...,"[11, 10, 10, 12, 14, 9, 8, 2, 1, 8, 12, 11, 1,...",12,"12,8,1","[1, 2, 3]",0.250000,8.000000,0,231,2485,9.295775
7,0,8,2170,1,1.0,0.00,3,AUS,Gelding,60,...,"[6, 7, 4, 6, 5, 10, 14, 6, 7, 6, 12, 8, 8, 5, ...",6,"6,7,5","[1, 2, 4]",0.210526,6.157895,1,61,246,24.796748
8,0,9,1730,13,0.0,9.75,3,NZ,Gelding,60,...,"[5, 11, 12, 8, 4, 6, 9, 7, 3, 5, 2, 1, 14, 5, ...",5,"5,11,7","[1, 2, 3]",0.227273,6.727273,0,143,1188,12.037037
9,0,10,2998,14,0.0,999.00,3,AUS,Mare,60,...,"[9, 8, 13, 14, 14]",14,"14,9,8","[8, 9, 13]",0.000000,11.600000,0,207,1104,18.750000


In [48]:
# Convert DataFrame to CSV for ranstab modeling
all_runs.to_csv('all_runs_processed_transtab.csv', index=False) 

In [21]:
# The below dataframe showing the statistic of all jockeys
# For example, jockey ID:3 has achieved top-three placements in 1811 out of 4719 races, 
# resulting in a placement percentage of 38.38%
jockey_stats.head(5)

,jockey_id,Top3Finishes,jockey_total_races,jockey_placement_perc
0,0,0,4,0.000000
1,1,19,136,13.970588
2,2,1811,4719,38.376775
3,3,27,118,22.881356
4,4,0,2,0.000000


In [22]:
# encode all the class into numerical value
# encode ordinal columns: config,going 
import sklearn.preprocessing as preprocessing

ordinal_encoder = preprocessing.OrdinalEncoder()
all_runs['config'] = ordinal_encoder.fit_transform(all_runs['config'].values.reshape(-1, 1))
all_runs['going'] = ordinal_encoder.fit_transform(all_runs['going'].values.reshape(-1, 1))
all_runs['horse_ratings'] = ordinal_encoder.fit_transform(all_runs['horse_ratings'].values.reshape(-1, 1))

# encode nominal column: venue, horse_country, horse_type
nominal_encoder = preprocessing.LabelEncoder()

all_runs['venue'] = nominal_encoder.fit_transform(all_runs['venue'])  # convert to 0/1
#all_runs['horse_country'] = nominal_encoder.fit_transform(all_runs['horse_country']) # 1, 1
all_runs['horse_type'] = nominal_encoder.fit_transform(all_runs['horse_type']) # 0,1,2,3.....,8,9

In [23]:
all_runs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79447 entries, 0 to 79446
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   race_id                     79447 non-null  int64  
 1   horse_no                    79447 non-null  int64  
 2   horse_id                    79447 non-null  int64  
 3   result                      79447 non-null  int64  
 4   won                         79447 non-null  float64
 5   lengths_behind              79447 non-null  float64
 6   horse_age                   79447 non-null  int64  
 7   horse_country               79447 non-null  object 
 8   horse_type                  79447 non-null  int64  
 9   horse_rating                79447 non-null  int64  
 10  horse_gear                  79447 non-null  int64  
 11  declared_weight             79447 non-null  float64
 12  actual_weight               79447 non-null  int64  
 13  draw                        794

In [24]:
# Convert DataFrame to CSV for modeling
all_runs.to_csv('all_runs_processed.csv', index=False) 

In [25]:
all_runs['horse_gear']

0        0
1        0
2        0
3        0
4        0
        ..
79442    1
79443    0
79444    1
79445    0
79446    0
Name: horse_gear, Length: 79447, dtype: int64